In [5]:
import sys
sys.path.append('/home/achint/Generative_Modelling/POISEVAE-master')
import os, copy, time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal, Laplace
from tqdm import tqdm
import matplotlib.pyplot as plt
import poisevae
from poisevae.utils import CategoricalImage as Categorical
from poisevae.datasets import MNIST_MNIST
from poisevae.networks.MNISTMNISTNetworks_X import EncMNIST1, DecMNIST1, EncMNIST2, DecMNIST2
from poisevae.networks.MNISTMNISTNetworks_pixelcnn import pixelcnn_decoder
# from poisevae.networks.conv_layers import MaskedConv2d, CroppedConv2d
from poisevae.networks.model import PixelCNN

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
MNIST_PATH = "/home/achint/datasets/MNIST/%s.pt"
SVHN_PATH = "/home/achint/datasets/SVHN/%s_32x32.mat"

joint_dataset_train = MNIST_MNIST.MNIST_MNIST(mnist_pt_path_1=MNIST_PATH % 'training', mnist_pt_path_2=MNIST_PATH % 'training')
joint_dataset_test  = MNIST_MNIST.MNIST_MNIST(mnist_pt_path_1=MNIST_PATH % 'test', mnist_pt_path_2=MNIST_PATH % 'test')

In [7]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(joint_dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(joint_dataset_test, batch_size=batch_size, shuffle=True, drop_last=True)
len(train_loader), len(test_loader)

(1875, 312)

In [8]:
lat1, lat2 = 20, 20
enc_mnist1 = EncMNIST1(lat1, lat2).to(device)
enc_mnist2 = EncMNIST2(lat1, lat2).to(device)

dec_mnist1 = pixelcnn_decoder(PixelCNN(lat1, 1)).to(device)
dec_mnist2 = pixelcnn_decoder(PixelCNN(lat1, 1)).to(device)

# Options: 'autograd' and 'gradient'
vae = poisevae.POISEVAE_Gibbs('gradient',
                              [enc_mnist1, enc_mnist2], [dec_mnist1, dec_mnist2], likelihoods=[Categorical, Categorical],
                              latent_dims=[lat1, (lat2)], enc_config='nu', KL_calc='derivative', 
                              batch_size=batch_size, reduction='mean'
                             ).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
writer = None

In [10]:
PATH = '/home/achint/runs'
epoch = 0
epochs = 30 + epoch
for epoch in tqdm(range(epoch, epochs)):
    poisevae.utils.train(vae, train_loader, optimizer, epoch, 3, 15, writer)
    poisevae.utils.test(vae, test_loader, epoch, 3, 15, writer)
    if (epoch+1) % 10 == 0 and epoch > 0:
        poisevae.utils.save_checkpoint(vae, optimizer, os.path.join(PATH, 'training_%d.pt' % (epoch+1)), epoch+1)


  0%|                                                                                                                                        | 0/30 [00:00<?, ?it/s]

mu max: 0.1352165937423706 mu mean: 0.049346815794706345
mup max: 0.16211947798728943 mup mean: 0.05548923835158348
var min: 0.8661730885505676 var mean: 0.9966581463813782
varp min: 0.8579937219619751 varp mean: 1.008426547050476


TypeError: forward() missing 2 required positional arguments: 'img' and 'generate_mode'